<a href="https://colab.research.google.com/github/aimat163/python/blob/main/ABC_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np
!pip install Faker
from faker import Faker
Faker = Faker('ru_RU')

# Генерируем синтетические данные о продажах

In [ ]:
df = pd.DataFrame({
    'item_name': [Faker.word() for i in range(100)], #название товара
    'count_sell_items': np.random.randint(10, 50, size=100), #сколько штук продали
    'zak_price': np.random.randint(500, 5000, size=100) #закупочная цена проданных товаров
})

df['roz_price'] = df['zak_price']*np.random.uniform(1.2, 1.5, size=100) #розничная цена проданных товаров

# Анализируем данные

In [ ]:
class ABCFullAnalyzer:
    def __init__(self, df):
        self.df = df.copy()
        self.df_count = None
        self.df_revenue = None
        self.df_margin = None
        self.df_final = None

    def abc_by_count(self): #ABC по продажам
        df = self.df.sort_values(by='count_sell_items', ascending=False).reset_index(drop=True)
        df['perc_kol'] = df['count_sell_items'] / df['count_sell_items'].sum() * 100
        df['cumsum_col'] = np.cumsum(df['perc_kol'])
        df['abc_col'] = df['cumsum_col'].apply(lambda x: 'A' if x < 80 else ('B' if x < 95 else 'C'))
        self.df_count = df
        return df

    def abc_by_revenue(self): #ABC по выручке
        df = self.df_count.sort_values(by='roz_price', ascending=False).reset_index(drop=True)
        df['perc_roz'] = df['roz_price'] / df['roz_price'].sum() * 100
        df['cumsum_roz'] = np.cumsum(df['perc_roz'])
        df['abc_roz'] = df['cumsum_roz'].apply(lambda x: 'A' if x < 80 else ('B' if x < 95 else 'C'))
        self.df_revenue = df
        return df

    def abc_by_margin(self): #ABC по маржинальности
        df = self.df_revenue.copy()
        df['margin'] = (df['roz_price'] - df['zak_price']) / df['roz_price']
        df = df.sort_values(by='margin', ascending=False).reset_index(drop=True)
        df['perc_margin'] = df['margin'] / df['margin'].sum() * 100
        df['cumsum_margin'] = np.cumsum(df['perc_margin'])
        df['abc_margin'] = df['cumsum_margin'].apply(lambda x: 'A' if x < 60 else ('B' if x < 90 else 'C'))
        self.df_margin = df
        return df

    def make_full_abc(self):
        df = self.df_margin.copy()
        df['ABC'] = df['abc_col'] + df['abc_roz'] + df['abc_margin']
        df = df.sort_values(by='ABC', ascending=True).reset_index(drop=True)
        self.df_final = df
        return df

    def run_full_analysis(self):
        self.abc_by_count()
        self.abc_by_revenue()
        self.abc_by_margin()
        return self.make_full_abc()

    def get_final(self):
        return self.df_final


In [ ]:
analyzer = ABCFullAnalyzer(df)

In [ ]:
analyzer.run_full_analysis()

,item_name,count_sell_items,zak_price,roz_price,perc_kol,cumsum_col,abc_col,perc_roz,cumsum_roz,abc_roz,margin,perc_margin,cumsum_margin,abc_margin,ABC
0,приличный,25,4156,5596.360898,0.869868,74.286708,A,1.576302,31.727740,A,0.257375,1.012120,58.558528,A,AAA
1,дремать,28,4516,6229.489534,0.974252,67.884482,A,1.754632,16.828611,A,0.275061,1.081671,47.087034,A,AAA
2,помимо,28,4621,6705.672573,0.974252,63.013222,A,1.888757,7.770446,A,0.310882,1.222536,24.062184,A,AAA
3,белье,43,4926,7154.147862,1.496173,28.914405,A,2.015077,2.015077,A,0.311448,1.224764,22.839648,A,AAA
4,необычный,40,3637,5216.286542,1.391788,37.439109,A,1.469248,40.792473,A,0.302761,1.190600,31.272962,A,AAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,господь,13,2998,3917.946389,0.452331,96.868476,C,1.103550,62.348760,A,0.234803,0.923358,67.138884,B,CAB
96,полностью,13,3551,4447.406515,0.452331,95.963814,C,1.252681,55.533754,A,0.201557,0.792619,89.191475,B,CAB
97,заработать,12,4036,5231.758271,0.417537,97.703549,C,1.473606,39.323225,A,0.228558,0.898798,70.774949,B,CAB
98,коллектив,14,2441,3160.405910,0.487126,95.059151,C,0.890177,77.142025,A,0.227631,0.895153,71.670102,B,CAB
